# OpenMC Learning
This notebook is for me to learn how to use OpenMC. The first problem is a sphere of radius 20 cm 
made of U-235 with 10,000 particles to calculate the flux at the surface of the sphere.

In [1]:
import numpy as np
import matplotlib.pyplot
import openmc

## Problem 1

In [28]:
#Add Fe-56
materials = openmc.Materials()

u235 = openmc.Material()
u235.add_nuclide('U235', 1, 'ao')
u235.set_density('g/cm3', 19.1)
u235.temperature = 900

materials.append(u235)

In [29]:
# Defining geometry
geometry = openmc.Geometry()

sphere = openmc.Sphere(r=20)
inside_sphere = -sphere
outside_sphere = +sphere
sphere.boundary_type = 'vacuum'

cell1 = openmc.Cell(fill=u235, region=inside_sphere)
cell2 = openmc.Cell(region=outside_sphere)
universe = openmc.Universe(cells=[cell1, cell2])

geometry.root_universe = universe

In [30]:
settings = openmc.Settings()
settings.run_mode = 'eigenvalue'
settings.particles = 10000
settings.generations_per_batch = 10
settings.batches = 100
settings.inactive = 5

In [19]:
tally = openmc.Tally(1)
tally.filters = [openmc.CellFilter(cell1)]
tally.scores = ['total']
tallies = openmc.Tallies([tally])

/home/abachmann/anaconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Tally instance already exists with id=1.
  warn(msg, IDWarning)


In [31]:
materials.export_to_xml()
geometry.export_to_xml()
settings.export_to_xml()
tallies.export_to_xml()

#openmc.run()

## Problem 2
This problem performs depletion

In [2]:
import openmc.deplete as od

In [3]:
model = openmc.Model(geometry = geometry, materials = materials, settings = settings)

NameError: name 'geometry' is not defined

In [36]:
model

In [37]:
settings

In [23]:
microsxs = od.MicroXS.from_model(model, u235, chain_file="chain_endfb71_pwr.xml")

/home/abachmann/anaconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=4.
  warn(msg, IDWarning)
/home/abachmann/anaconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=5.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Reading Kr85 from /home/abachmann/xsdata/mcnp_endfb71/Kr85.h5
 Reading Kr86 from /home/abachmann/xsdata/mcnp_endfb71/Kr86.h5
 Reading Rb85 from /home/abachmann/xsdata/mcnp_endfb71/Rb85.h5
 Reading Rb86 from /home/abachmann/xsdata/mcnp_endfb71/Rb86.h5
 Reading Rb87 from /home/abachmann/xsdata/mcnp_endfb71/Rb87.h5
 Reading Sr84 from /home/abachmann/xsdata/mcnp_endfb71/Sr84.h5
 Reading Sr86 from /home/abachmann/xsdata/mcnp_endfb71/Sr86.h5
 Reading Sr87 from /home/abachmann/xsdata/mcnp_endfb71/Sr87.h5
 Reading Sr88 from /home/abachmann/xsdata/mcnp_endfb71/Sr88.h5
 Reading Sr89 from /home/abachmann/xsdata/mcnp_endfb71/Sr89.h5
 Reading Sr90 from /home/abachmann/xsdata/mcnp_endfb71/Sr90.h5
 Reading Y89 from /home/abachmann/xsdata/mcnp_endfb71/Y89.h5
 Reading Y90 from /home/abachmann/xsdata/mcnp_endfb71/Y90.h5
 Reading Y91 from /home/abachmann/xsdata/mcnp_endfb71/Y91.h5
 Reading Zr90 from /home/abachmann/xsdata/mcnp_endfb71/Zr90.h5
 Reading Zr91 from /home/abachmann/xsdata/mcnp_endfb71/Zr91.h

 Reading Ba140 from /home/abachmann/xsdata/mcnp_endfb71/Ba140.h5
 Reading La138 from /home/abachmann/xsdata/mcnp_endfb71/La138.h5
 Reading La139 from /home/abachmann/xsdata/mcnp_endfb71/La139.h5
 Reading La140 from /home/abachmann/xsdata/mcnp_endfb71/La140.h5
 Reading Ce136 from /home/abachmann/xsdata/mcnp_endfb71/Ce136.h5
 Reading Ce138 from /home/abachmann/xsdata/mcnp_endfb71/Ce138.h5
 Reading Ce139 from /home/abachmann/xsdata/mcnp_endfb71/Ce139.h5
 Reading Ce140 from /home/abachmann/xsdata/mcnp_endfb71/Ce140.h5
 Reading Ce141 from /home/abachmann/xsdata/mcnp_endfb71/Ce141.h5
 Reading Ce142 from /home/abachmann/xsdata/mcnp_endfb71/Ce142.h5
 Reading Ce143 from /home/abachmann/xsdata/mcnp_endfb71/Ce143.h5
 Reading Ce144 from /home/abachmann/xsdata/mcnp_endfb71/Ce144.h5
 Reading Pr141 from /home/abachmann/xsdata/mcnp_endfb71/Pr141.h5
 Reading Pr142 from /home/abachmann/xsdata/mcnp_endfb71/Pr142.h5
 Reading Pr143 from /home/abachmann/xsdata/mcnp_endfb71/Pr143.h5
 Reading Nd142 from /home

 Reading U231 from /home/abachmann/xsdata/mcnp_endfb71/U231.h5
 Reading U232 from /home/abachmann/xsdata/mcnp_endfb71/U232.h5
 Reading U233 from /home/abachmann/xsdata/mcnp_endfb71/U233.h5
 Reading U234 from /home/abachmann/xsdata/mcnp_endfb71/U234.h5
 Reading U236 from /home/abachmann/xsdata/mcnp_endfb71/U236.h5
 Reading U237 from /home/abachmann/xsdata/mcnp_endfb71/U237.h5
 Reading U238 from /home/abachmann/xsdata/mcnp_endfb71/U238.h5
 Reading U239 from /home/abachmann/xsdata/mcnp_endfb71/U239.h5
 Reading U240 from /home/abachmann/xsdata/mcnp_endfb71/U240.h5
 Reading U241 from /home/abachmann/xsdata/mcnp_endfb71/U241.h5
 Reading Np234 from /home/abachmann/xsdata/mcnp_endfb71/Np234.h5
 Reading Np235 from /home/abachmann/xsdata/mcnp_endfb71/Np235.h5
 Reading Np236 from /home/abachmann/xsdata/mcnp_endfb71/Np236.h5
 Reading Np237 from /home/abachmann/xsdata/mcnp_endfb71/Np237.h5
 Reading Np238 from /home/abachmann/xsdata/mcnp_endfb71/Np238.h5
 Reading Np239 from /home/abachmann/xsdata/mc

...

       29/1    1.72845    1.73217 +/- 0.00102
       29/2    1.72257    1.73213 +/- 0.00102
       29/3    1.75439    1.73222 +/- 0.00102
       29/4    1.72639    1.73220 +/- 0.00102
       29/5    1.74753    1.73226 +/- 0.00101
       29/6    1.72720    1.73224 +/- 0.00101
       29/7    1.72383    1.73221 +/- 0.00101
       29/8    1.74129    1.73225 +/- 0.00100
       29/9    1.74015    1.73228 +/- 0.00100
      29/10    1.71227    1.73220 +/- 0.00100
       30/1    1.74993    1.73227 +/- 0.00100
       30/2    1.71140    1.73218 +/- 0.00100
       30/3    1.72002    1.73213 +/- 0.00099
       30/4    1.74539    1.73219 +/- 0.00099
       30/5    1.74139    1.73222 +/- 0.00099
       30/6    1.74904    1.73229 +/- 0.00099
       30/7    1.75781    1.73240 +/- 0.00099
       30/8    1.73426    1.73240 +/- 0.00098
       30/9    1.75939    1.73251 +/- 0.00099
      30/10    1.74248    1.73255 +/- 0.00098
       31/1    1.73439    1.73256 +/- 0.00098
       31/2    1.73785    1.73258 

      46/10    1.73685    1.73294 +/- 0.00076
       47/1    1.74498    1.73297 +/- 0.00075
       47/2    1.74293    1.73299 +/- 0.00075
       47/3    1.72903    1.73299 +/- 0.00075
       47/4    1.74415    1.73301 +/- 0.00075
       47/5    1.74308    1.73304 +/- 0.00075
       47/6    1.72891    1.73303 +/- 0.00075
       47/7    1.72856    1.73302 +/- 0.00074
       47/8    1.71890    1.73298 +/- 0.00074
       47/9    1.72232    1.73296 +/- 0.00074
      47/10    1.71730    1.73292 +/- 0.00074
       48/1    1.72416    1.73290 +/- 0.00074
       48/2    1.72417    1.73288 +/- 0.00074
       48/3    1.73718    1.73289 +/- 0.00074
       48/4    1.74550    1.73292 +/- 0.00074
       48/5    1.71408    1.73287 +/- 0.00074
       48/6    1.74155    1.73289 +/- 0.00073
       48/7    1.75258    1.73294 +/- 0.00073
       48/8    1.72053    1.73291 +/- 0.00073
       48/9    1.73157    1.73291 +/- 0.00073
      48/10    1.74630    1.73294 +/- 0.00073
       49/1    1.74622    1.73297 

       64/9    1.71155    1.73251 +/- 0.00065
      64/10    1.73368    1.73251 +/- 0.00064
       65/1    1.73633    1.73252 +/- 0.00064
       65/2    1.72500    1.73250 +/- 0.00064
       65/3    1.72447    1.73249 +/- 0.00064
       65/4    1.76505    1.73254 +/- 0.00064
       65/5    1.74977    1.73257 +/- 0.00064
       65/6    1.72793    1.73257 +/- 0.00064
       65/7    1.76571    1.73262 +/- 0.00064
       65/8    1.75512    1.73266 +/- 0.00064
       65/9    1.74489    1.73268 +/- 0.00064
      65/10    1.73773    1.73269 +/- 0.00064
       66/1    1.73020    1.73268 +/- 0.00064
       66/2    1.68916    1.73261 +/- 0.00064
       66/3    1.75913    1.73266 +/- 0.00064
       66/4    1.76326    1.73271 +/- 0.00064
       66/5    1.74395    1.73272 +/- 0.00064
       66/6    1.75162    1.73276 +/- 0.00064
       66/7    1.74886    1.73278 +/- 0.00064
       66/8    1.72727    1.73277 +/- 0.00064
       66/9    1.71548    1.73274 +/- 0.00064
      66/10    1.72176    1.73273 

       82/8    1.73866    1.73274 +/- 0.00057
       82/9    1.71969    1.73272 +/- 0.00057
      82/10    1.74149    1.73273 +/- 0.00057
       83/1    1.73110    1.73273 +/- 0.00057
       83/2    1.74051    1.73274 +/- 0.00057
       83/3    1.74252    1.73275 +/- 0.00057
       83/4    1.71032    1.73273 +/- 0.00057
       83/5    1.74511    1.73274 +/- 0.00057
       83/6    1.73440    1.73274 +/- 0.00057
       83/7    1.72999    1.73274 +/- 0.00057
       83/8    1.71500    1.73272 +/- 0.00057
       83/9    1.72293    1.73270 +/- 0.00056
      83/10    1.71772    1.73269 +/- 0.00056
       84/1    1.72823    1.73268 +/- 0.00056
       84/2    1.75223    1.73270 +/- 0.00056
       84/3    1.75522    1.73273 +/- 0.00056
       84/4    1.72301    1.73272 +/- 0.00056
       84/5    1.72697    1.73271 +/- 0.00056
       84/6    1.71148    1.73269 +/- 0.00056
       84/7    1.75026    1.73271 +/- 0.00056
       84/8    1.71472    1.73269 +/- 0.00056
       84/9    1.74842    1.73271 

      100/7    1.74351    1.73283 +/- 0.00051
      100/8    1.72544    1.73282 +/- 0.00051
      100/9    1.74078    1.73283 +/- 0.00051
     100/10    1.71819    1.73281 +/- 0.00051
 Creating state point statepoint.100.h5...

 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 7.0569e+01 seconds
   Reading cross sections          = 7.0560e+01 seconds
 Total time in simulation          = 3.9463e+03 seconds
   Time in transport only          = 3.9445e+03 seconds
   Time in inactive batches        = 4.8471e+01 seconds
   Time in active batches          = 3.8978e+03 seconds
   Time synchronizing fission bank = 1.0309e+00 seconds
     Sampling source sites         = 8.7341e-01 seconds
     SEND/RECV source sites        = 1.5089e-01 seconds
   Time accumulating tallies       = 2.8350e-01 seconds
   Time writing statepoints        = 2.4216e-02 seconds
 Total time for finalization       = 2.9298e-03 seconds
 Total time elapsed   

In [25]:
ind_op = od.IndependentOperator(materials, microsxs, chain_file = "chain_endfb71_pwr.xml")

RuntimeError: Volume not specified for depletable material with ID=3.

In [26]:
materials

[Material
 	ID             =	3
 	Name           =	
 	Temperature    =	900
 	Density        =	19.1 [sum]
 	S(a,b) Tables  
 	Nuclides       
 	U235           =	0.048936762332792594 [ao]
 	H1             =	1e-21        [ao]
 	H2             =	1e-21        [ao]
 	H3             =	1e-21        [ao]
 	He3            =	1e-21        [ao]
 	He4            =	1e-21        [ao]
 	Li6            =	1e-21        [ao]
 	Li7            =	1e-21        [ao]
 	Be7            =	1e-21        [ao]
 	Be9            =	1e-21        [ao]
 	B10            =	1e-21        [ao]
 	B11            =	1e-21        [ao]
 	N14            =	1e-21        [ao]
 	N15            =	1e-21        [ao]
 	O16            =	1e-21        [ao]
 	O17            =	1e-21        [ao]
 	F19            =	1e-21        [ao]
 	Na22           =	1e-21        [ao]
 	Na23           =	1e-21        [ao]
 	Mg24           =	1e-21        [ao]
 	Mg25           =	1e-21        [ao]
 	Mg26           =	1e-21        [ao]
 	Al27           =	1e-21        [ao]
 